<a href="https://colab.research.google.com/github/andrefs/mapi-faml-proj/blob/main/code/faml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load stuff


## Libraries

In [ ]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

## Embeddings

In [ ]:
def normalize(word_vec):
    norm=np.linalg.norm(word_vec)
    if norm == 0: 
       return word_vec
    return word_vec/norm

embs = {}
with open('../2_clean_datasets/embeddings.txt', newline='') as embs_file:
    reader = csv.reader(embs_file, delimiter=' ')
    for line in reader:
        term = line[0].replace('http://dbpedia.org/resource/' , '')
        vector = normalize([float(x) for x in line[1:]])
        embs[term] = vector
        
len(embs.keys())

## Relatedness pairs

In [ ]:
relness = {}
with open('../2_clean_datasets/relatedness.tsv', newline='') as rel_file:
    lc = 0
    reader = csv.reader(rel_file, delimiter='\t')
    for line in reader:
        lc += 1
        t1 = line[0].replace('http://dbpedia.org/resource/' , '')
        t2 = line[1].replace('http://dbpedia.org/resource/' , '')
        relness[t1] = relness.get(t1,{})
        relness[t1][t2] = [float(line[2]), line[3]]

lc